# **Ema's Quantum Computer**

https://www.hackerrank.com/challenges/two-pluses/problem?isFullScreen=true

## **Problem description**: 

Ema has a grid of good 'G' and bad 'B' cells of size m * n. Find the largest product of the volume of two non-overlapping plus signs. A plus sign is a symmetric plus sign of width 1 with all arms of the same length and a centerpoint at which all four arms meet.

For notation simplicity, m will be replaced by n.


## Here I present an O(n^2) fast encoding algorithm with an O(n^2.5) fast spatial comparison algorithm 

A list of the most notable tools available:
- Dictionary Indexing
- Chebyshev distance spatial sorting
- Key sorting and greedy algorithms
- Early stopping heuristics
- Spatially bounded algorithms that reduce O(n^2) permutations into O(n) linear reads
- Graham scan method of creating convex hulls.

## **Step 1**: O(n^2) runtime in dictionary encoding of pluses

A naive implementation of encoding pluses searches through every (x, y) coordinate and indexes for pluses, taking O(n^3) to search.

This can be done faster by a factor of n in the following manner:
    - parse through each row and for each cell record the length of the longest line segment of 'G' cells and the position of the rightmost cell of the line segment
    - parse through each colum n and for each cell, use the line segment features to compute the centerpoint and volume of the largest plus at (x, y)

Total runtime of search and encoding: O(n^2)

## **Python Class: Plus** -- Data structure for logical simplicity: 
- Keep a dictionary of pluses with coordinate keys
- Keep a second dictionary of the pluses sorted by volume
    - There at most min(n, m) different size pluses
    - Heuristic sort with chebyshev distance from origin -- gives higher chance of early stopping -- O(n^2 log n) runtime
- Logically sort the lengths of the pluses to search with early stopping heuristics

## **Step 2**: Finding the largest product.

- Total pluses: n^2
- Total unique volumes: n

A naive implementation brute forces the combinations of every two plus signs, taking O(n^4) to search.

**O(n^4) algorithm with preprocessing, heuristics, and early stopping implemented at the bottom with python.**

## Bounding Box Algorithm:

**Objective**: Implement an efficient solution for computing the maximal product of overlapping plus signs on a grid using bounding boxes and convex hulls.

### Algorithm Description

- **Initial Setup**:
    - We are given an array of pluses of equal size. We construct a bounding box such that any plus center within the box intersect with other pluses. Plus centers outside this box are necessarily distinct, significantly reducing initial computation.

- **Overlap Handling**:
    - Pluses of equal size must overlap twice when they intersect. This scenario is efficiently handled through direct computation:
    - **Case 1**: For overlaps along the x-axis (x_diff == 0) or y-axis (y_diff == 0), the maximum product is computed by equally dividing the arm length between the two intersecting plus signs. This can be computed in constant time.
    - **Case 2**: For diagonal intersections (x_diff == y_diff), the maximum product is achieved by trimming one plus until there is no intersection.
    - **Case 3**: For other cases, the maximum product is computed as (max(x_diff, y_diff) * 4 + 1) ^ 2. The calculation involves trimming both pluses until they no longer overlap.

- **Convex Hull Application**:
    - **Preprocessing**: A preprocessing sort of O(n^2 log n) is employed to arrange points for convex hull construction using the Graham scan method.
    - **Hull Construction**:
        - Convex hulls are constructed for each x and y coordinate, with points removed strategically (dropouts) to reduce overlap and ensure comprehensive coverage.
        - Each convex hull can have at most n extremal points due to the reduced dimensionality of the problem.
        - Two odds and evens convex hulls are constructed by removing odd and even indexed extremal points in batches. This method ensures minimal overlap and comprehensive coverage. This, combined with a convex hull without dropouts comprehensively covers all the possible options.
    - **Spatial Efficiency**:
        - The combined spatial complexity of constructing and storing these convex hulls is O(n), due to the overlapping information and the linear nature of the construction process.

        - **Proof**:
        1. **Definition**: A convex hull is a one-dimensional boundary that encloses a set of points on a 2D plane, forming the minimal convex set (Axiomatic).
        2. **Extremal Points Limit**: By definition, the convex hull of a set of n^2 points on a plane can have at most n extremal points (Derived from 1).
        3. **Lesser Convex Hulls**: When any point is excluded from the grid, a new convex hull formed has no more than n extremal points (Consequence of 2).
        4. **Distinct Extremal Points**: Removing points that are two units apart results in convex hulls with distinct new extremal points, except at a shared extremal point (Geometric property from 1).
        5. **Batch Processing**: Constructing two distinct convex hulls by alternately removing odd and even-indexed points ensures all scenarios are covered with minimal overlap (Extension of 3 and 4).
        6. **Coverage**: These two lesser distinct convex hulls collectively with the global convex hull (no dropouts) represent all potential configurations arising from the individual removal of any point (Geometric property of 4, 5).
        7. **Information Content**: The total informational content (number of unique extremal points) within these hulls is O(n), reflecting linear complexity (Upper bound from 2).
        8. **Construction Complexity**: Building these convex hulls requires linear time, O(n), due to the limited number of points processed (Efficiency derived from iterative approach in 4, 5, and 7).
        9. **Conclusion**: Considering all individual convex hulls constructed by excluding each point, the entire dataset exhibits linear complexity in both construction and informational content (Final conclusion from 8).

        - **Conclusion**: This proof demonstrates that handling convex hulls derived by excluding points from a grid is linear complexity, O(n). This property significantly enhances the efficiency of any algorithm requiring geometric boundary analyses in grid-based data structures.

    - **Example**: given a plus at (3, 3) with volume 9, a plus at (4,6) is identical to a plus at (5,6) because the optimal solution for both is to truncate to the largest of the x and y differences. Notable exceptions are overlapping pluses at (3, 3) and (3, 4) as well as directly diagonal pluses (3, 3) and (4, 4) which can be handled separately through a number of sweep line algorithms horizontally, vertically, and diagonally.

**Conclusion**

This bounding box algorithm provides an efficient method for handling overlapping geometries in grid-based data structures, significantly enhancing algorithmic efficiency through geometric insights and spatial indexing. This proof demonstrates the algorithm's effectiveness in reducing computational complexity to a linear scale.


## **Final step**: Permutation between all n distinct volumes

### **Current steps taken so far**:
- n^2 total pluses recorded in O(n^2) time
- O(n^2 * log(n)) sort along one axis for each volume for Graham scan
- n different volumes of pluses
    - n different bounding boxes created in O(n^2) time
    - n different convex hulls of size O(n) created in O(n^2) time globally -- see above bounding box algorithm
- **Total**: O(n^2 * log(n))

### **Permutation**: The spatial comparison algorithm
- Comparing two convex hulls for volume V1 V2 plus volumes where V1 > V2: (n choose 2) = n^2 comparisons -- runtime O(n^2 * comparison time)

    - 2 bounding boxes
    - 2 convex hulls
        - n elements per convex hull

    - Shrink the bounding box of V1 to match V2 by recomputing V1 as if all the pluses were of V2 volume and take a sorted permutation: Convex hull might go outside bounding box.
        - **Case 1**: Some convex hull plus centerpoints fall outside of the opposing bounding box
            - At minimum, we have V2 * V2 = max product
            - A final permuntation takes at most O(n^2) permutations -- convex hulls have O(n) spatial complexity
            - Search is halted
        - **Case 2**: Some conves hull plus centerpoints fall outside of its own shrunk bounding box
            - At minimum, we have V2 * V2 = max product
            - A final permuntation takes at most O(n^2) permutations
            - Search is halted
        - **Case 3**: Both convex hulls fall inside the overlap of both convex boxes
            - In the worst case, the maximum number of pluses remaining inside convex hulls exists when pluses are evenly distributed across volumes. That leaves sqrt(n) items in n bins or O(n^1.5) spatial complexity.
            - **A brute force permutation of the remaining bins yields (n^1.5 choose 2) or n^3 global runtime**
            - Given that all plus centers in case 1 lie within the bounding box of V2, that means all pluses intersect at two points. Thus, the **chebyshev distance** holds and the search dimension is linear in runtime and more informative than a 2d graham scan.
            - **Comparison algorithm**:

                **Optimizing Spatial Analysis with Chebyshev Distance Convex Hulls:**

                To efficiently handle spatial relationships between pluses on a 2D grid, we employ a method utilizing Chebyshev distance convex hulls with row and column drop outs. This approach involves focusing on the extremal distances from each corner of a predefined bounding box.

                1. **Chebyshev Distance Calculation**:
                - For each plus sign, calculate the Chebyshev distance to each of the four corners of the bounding box. The Chebyshev distance is determined by the maximum of the horizontal (x) and vertical (y) differences between the plus sign and the corner.
                
                2. **Maintaining Convex Hulls**:
                - If this is confusing, it is identical to a "max list" keeping track of the current list of max chebyshev distance points, one max list for each corner of the bounding box.
                - Create four "partial" convex hulls, one for each corner of the bounding box. Each convex hull tracks the plus signs that represent the maximum Chebyshev distance from its respective corner.
                - Whenever a new plus sign has a larger Chebyshev distance than the current maximum for a particular corner, update that corner's convex hull by replacing the existing points with this new point. This ensures that each convex hull always represents the furthest points along the Chebyshev metric.
                
                3. **Efficient Data Management**:
                - These convex hulls are 1-dimensional in nature, reducing complexity. As we only need to maintain the maximum distance points, the update operation is simplified to checking and updating maximum values, which can be done in constant time.
                - This method requires minimal data structures, primarily four variables to store the maximum distance pluses for each corner. Updates involve simple comparisons and assignments, leveraging the 1D nature of the hulls to ensure efficiency.

                4. **Algorithmic Impact**:
                - This strategy allows for rapid determination of spatial relationships and overlaps between pluses, essential for calculating potential interactions and intersections efficiently.
                - By focusing only on the points with the maximum Chebyshev distances, we significantly reduce the computational overhead, as we limit the number of points to consider in detailed intersection or overlap checks.

            - Assuming worst case for permutations globally, that each volume has sqrt(n) extremal points on the convex hull:
            - **Comparison time**: Linear runtime over O(sqrt(n)) pluses --> **O(sqrt(n))**
            - **Global complexity**: O(n^2 * comparison time) --> O(n^2 * sqrt(n)) --> **O(n^2.5)**
    
**Final runtime: O(n^2.5)**

In [23]:
def twoPluses(grid, print_pluses = True, printpluses_min_volume = 2):
    class Plus:
        def __init__(self):
            self.pluses = {}
            self.decr_plus = {}
            self._maxprod = 0
            self.sorted_at_volume = set()
            self.upper_bound = None

        # Populate class attributes when inserting a plus
        def add(self,row,col,volume):
            t = (row, col)
            self.pluses[t] = volume
            if volume not in self.decr_plus:
                self.decr_plus[volume] = []
            self.decr_plus[volume].append(t)

        # For use in exists conditions
        def __contains__(self, row_col_tuple):
            return row_col_tuple in self.pluses
        
        # Retrieve pluses at specific volume
        def pluses_at_volume(self, volume):
            if volume not in self.sorted_at_volume:
                self.decr_plus[volume].sort(key = lambda x: x[0] + x[1])  # L1 norm very likely to have disjoint pluses.
                self.sorted_at_volume.add(volume)
            return self.decr_plus[volume]
        
        # Maxprod attribute custom handled to reduce clutter
        @property
        def maxprod(self):
            return self._maxprod
        @maxprod.setter
        def maxprod(self, value):
            if self._maxprod < value:
                self._maxprod = value
            else:
                pass
                # print(f'{value} too small to set as maxprod: {self.maxprod}')

        # Main algorithm to compute max product after populating
        def find_max(self):
            vol_desc = sorted(list(self.decr_plus.keys()),reverse=True)
            num_unique_volumes = len(vol_desc)
            for i in range(num_unique_volumes):
                for j in range(i+1):
                    
                    vol_i, vol_j = vol_desc[i], vol_desc[j]
                    
                    # Calculate ideal product without any interference
                    ideal_prod = vol_i * vol_j

                    # Case: Same size plus comparison                    
                    if i == j:
                        pluses = self.pluses_at_volume(vol_desc[i])
                        for index_a in range(len(pluses)):
                            for index_b in range(index_a + 1, len(pluses)):
                                if ideal_prod <= self.maxprod:
                                    break
                                self.find_prod(pluses[index_a], pluses[index_b], vol_i, vol_j, ideal_prod)
                    # Case: Different size plus comparison (two arrays)
                    else:
                        i_pluses = self.pluses_at_volume(vol_desc[i])
                        j_pluses = self.pluses_at_volume(vol_desc[j])
                        for iplus in i_pluses:
                            for jplus in j_pluses:
                                if ideal_prod <= self.maxprod:
                                    break
                                self.find_prod(iplus, jplus, vol_i, vol_j, ideal_prod)

            return self.maxprod

        def find_prod(self, a, b, vol_a, vol_b, ideal_prod):           
            # Calculate arm lengths for both pluses
            arm1 = (vol_a-1)//4
            arm2 = (vol_b-1)//4
            
            # Immediate return if centers coincide
            if a == b: 
                return
            
            # Calculate absolute positional differences
            x = abs(a[0]-b[0])
            y = abs(a[1]-b[1])
            
            # Check for direct line overlap without displacement
            if not x or not y:
                combined_arms = arm1 + arm2
                max_distance = max(x, y)
                if combined_arms < max_distance:
                    self.maxprod = ideal_prod
                    return
                else:
                    space = max_distance - 1
                    arm_small = min(arm1, arm2, space//2)
                    self.maxprod = (arm_small * 4 + 1) * ((space - arm_small) * 4 + 1)
                    return

            # Prepare zero-based indices for conflict checks
            x_space = x - 1
            y_space = y - 1
            
            # Define and check conflict conditions
            axbyconflict = (arm1 >= x and arm2 >= y)
            aybxconflict = (arm1 >= y and arm2 >= x)
            if not axbyconflict and not aybxconflict:  # No conflict
                self.maxprod = ideal_prod
            elif axbyconflict and aybxconflict:  # Symmetric error, two possibilities
                self.maxprod = max((min(x_space, y_space) * 4 + 1) * max(vol_a, vol_b), (max(x_space, y_space) * 4 + 1) ** 2)
            elif axbyconflict:  # Only AXBY conflict, require permutation
                self.maxprod = max(((x_space * 4 + 1) * vol_b), (vol_a * (y_space * 4 + 1)))
            else:  # Only AYBX conflict
                self.maxprod = max(((y_space * 4 + 1) * vol_b), (vol_a * (x_space * 4 + 1)))

        # Print method to visualize pluses
        def print(self,n = 1):
            print(f'printing pluses with volume {n} or larger')
            i = 0
            for key, value in self.pluses.items():
                if value >= n:
                    print(key,value,end=' ')
                    i +=1
                    if i % 8 ==  0: print()
            print(f'count {i}')
            
    # Main function to populate the dictionary of largest pluses at each row and column and their volumes
    def populate(grid):
        def process_segment(row, start, end, d):
            L = end - start + 1
            for col in range(start, end + 1):
                d[(row, col)] = (end, L)  # Populate flat dictionary
                
        def process_plus(col, start, end, d):
            L = end - start + 1
            for row in range(start, end + 1):
                row_right, row_L = d[(row, col)]
                col_right, col_L = end, L
                row_left, col_left = row_right - row_L + 1, col_right - col_L + 1
                volume = min((col - row_left), (row_right - col), (row - col_left), (col_right - row)) * 4 + 1
                dplus.add(row, col, volume)
        def parse_rows(grid):
            for rownum, row in enumerate(grid):
                start_i = None
                for i, val in enumerate(row):
                    if val == 'G':
                        if start_i is None:
                            start_i = i  # Start of a new plus
                    else:
                        if start_i is not None:  # End of a plus
                            process_segment(rownum, start_i, i - 1, d)
                            start_i = None
                if start_i is not None:  # Process any segment extending to the end of the row
                    process_segment(rownum, start_i, len(row) - 1, d)

        def parse_columns(grid):
                num_rows = len(grid)
                num_cols = len(grid[0])
                for col in range(num_cols):
                    start_i = None
                    for i in range(num_rows):
                        if grid[i][col] == 'G':
                            if start_i is None:
                                start_i = i
                        else:
                            if start_i is not None:    # end of a plus
                                process_plus(col, start_i, i - 1, d)
                                start_i = None
                    if start_i is not None:    # process any segment extending to end of col
                        process_plus(col, start_i, num_rows - 1, d)
        dplus = Plus()
        d = {}
        parse_rows(grid)
        parse_columns(grid)

        return dplus
    if not grid or len(grid[0]) == 0:
        return 0

    dplus = populate(grid)
    if print_pluses:
        dplus.print(printpluses_min_volume)
    return dplus.find_max()


In [33]:
def visualize_easier(grid):
    grid2 = []
    for row in grid:
        temp = row.replace('G','O')
        grid2.append(temp.replace('B','-'))
    for i in grid2:
        print(i)

from random import choice
def create_grid(n=10, m=10, dense = False, visualize = True):
    tiles = ('G', 'B')
    if dense:
        tiles = ('G',)
    grid = []
    for i in range(n):
        grid.append(''.join([choice(tiles) for _ in range(m)]))
    if visualize:
        visualize_easier(grid)
    return grid

In [34]:
import time

def measure_runtime(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()  # Record the start time
        result = func(*args, **kwargs)  # Execute the function
        end_time = time.time()  # Record the end time
        runtime = end_time - start_time  # Calculate the runtime
        print(f"Function {func.__name__} took {runtime:.6f} seconds to complete.")
        return result
    return wrapper
twoPluses_timed = measure_runtime(twoPluses)


In [35]:
grid = [
    'GGGGGGGGGGGG',
    'BGBGGGBGBGBG',
    'GGGGGGGGGGGG',
    'BGBGGGBGBGBG',
    'GGGGGGGGGGGG',
    'GGGGGGGGGGGG',
    'GGGGGGGGGGGG',
    'GGGGGGGGGGGG',
    'BGBGGGBGBGBG',
    'BGBGGGBGBGBG',
    'BGBGGGBGBGBG',
    'BGBGGGBGBGBG',
    'GGGGGGGGGGGG',
    'GGGGGGGGGGGG'
]
visualize_easier(grid)
print(f'max product: {twoPluses_timed(grid)}')

OOOOOOOOOOOO
-O-OOO-O-O-O
OOOOOOOOOOOO
-O-OOO-O-O-O
OOOOOOOOOOOO
OOOOOOOOOOOO
OOOOOOOOOOOO
OOOOOOOOOOOO
-O-OOO-O-O-O
-O-OOO-O-O-O
-O-OOO-O-O-O
-O-OOO-O-O-O
OOOOOOOOOOOO
OOOOOOOOOOOO
printing pluses with volume 2 or larger
(2, 1) 5 (4, 1) 5 (5, 1) 5 (6, 1) 5 (7, 1) 5 (12, 1) 5 (5, 2) 5 (6, 2) 5 
(2, 3) 9 (4, 3) 13 (5, 3) 13 (6, 3) 13 (7, 3) 13 (12, 3) 5 (1, 4) 5 (2, 4) 9 
(3, 4) 5 (4, 4) 17 (5, 4) 17 (6, 4) 17 (7, 4) 17 (8, 4) 5 (9, 4) 5 (10, 4) 5 
(11, 4) 5 (12, 4) 5 (2, 5) 9 (4, 5) 17 (5, 5) 21 (6, 5) 21 (7, 5) 21 (12, 5) 5 
(5, 6) 5 (6, 6) 5 (2, 7) 9 (4, 7) 17 (5, 7) 17 (6, 7) 17 (7, 7) 17 (12, 7) 5 
(5, 8) 5 (6, 8) 5 (2, 9) 9 (4, 9) 9 (5, 9) 9 (6, 9) 9 (7, 9) 9 (12, 9) 5 
(5, 10) 5 (6, 10) 5 count 50
Function twoPluses took 0.000000 seconds to complete.
max product: 189


In [36]:
grid = [
    'GBBBBBBGGGBGGBB',
    'GBBBBBBGGGBGGBB',
    'GBBBBBBGGGBGGBB',
    'GBBBBBBGGGBGGBB',
    'GGGGGGGGGGGGGGG',
    'GGGGGGGGGGGGGGG',
    'GBBBBBBGGGBGGBB',
    'GBBBBBBGGGBGGBB',
    'GGGGGGGGGGGGGGG',
    'GBBBBBBGGGBGGBB',
    'GBBBBBBGGGBGGBB',
    'GGGGGGGGGGGGGGG',
    'GGGGGGGGGGGGGGG',
    'GBBBBBBGGGBGGBB'
]
visualize_easier(grid)
print(f'max product: {twoPluses_timed(grid)}')

O------OOO-OO--
O------OOO-OO--
O------OOO-OO--
O------OOO-OO--
OOOOOOOOOOOOOOO
OOOOOOOOOOOOOOO
O------OOO-OO--
O------OOO-OO--
OOOOOOOOOOOOOOO
O------OOO-OO--
O------OOO-OO--
OOOOOOOOOOOOOOO
OOOOOOOOOOOOOOO
O------OOO-OO--
printing pluses with volume 2 or larger
(4, 7) 17 (5, 7) 21 (8, 7) 21 (11, 7) 9 (12, 7) 5 (1, 8) 5 (2, 8) 5 (3, 8) 5 
(4, 8) 17 (5, 8) 21 (6, 8) 5 (7, 8) 5 (8, 8) 21 (9, 8) 5 (10, 8) 5 (11, 8) 9 
(12, 8) 5 (4, 9) 17 (5, 9) 21 (8, 9) 21 (11, 9) 9 (12, 9) 5 (4, 11) 13 (5, 11) 13 
(8, 11) 13 (11, 11) 9 (12, 11) 5 (4, 12) 9 (5, 12) 9 (8, 12) 9 (11, 12) 9 (12, 12) 5 
count 32
Function twoPluses took 0.000000 seconds to complete.
max product: 273


### Worst case fully dense grid:

In [37]:
grid = create_grid(n=80, m=80, dense=True, visualize=False)
print(f'max product: {twoPluses_timed(grid,print_pluses = False)}')

Function twoPluses took 0.452318 seconds to complete.
max product: 12393


## Randomized grids:

In [41]:
for i in range(4):
    grid = create_grid(n=30, m=30)
    print(f'max product: {twoPluses_timed(grid)}')
    print('************************************')

-OO-OO--O-O-OOO-OOOOO-OOOO----
-----O-O-OO-OOO-O-O---O-OOO-O-
OOO---O----OO--O--OO--O-OOO--O
-OOOO----OOOOO-O----O-O-OOOOO-
-O--OO--OO---O-O--OOO---OOO-OO
-O-O--O-O-OOOO-O-O--OO--OOOOO-
OOOOOO--O-OO--OOOO-----O----O-
OO---OO---------OOO-O-O--O----
-O--OOO-O---O-O--O----O-O----O
-----O---OOO-O-OOO-OO-O---O-O-
----OOOO-OOO-O-O-O--O------OOO
O--OOOO-OOOOO--OOO-O-OOOO-OOOO
OO----OOOO-O----O--O--OO-OOO--
O-OO-O---OOO--O-OOOO-OOO----O-
-O---O-O-O-OO-OOO--O-O-OOO-O-O
OO--O--OOOOOOOO-OO-O-OO-OOO--O
--O-OO--O--OO----O---OO---O-OO
---OOOO-O-O---O---OOO-O-O-OO-O
--O--OO--O-O-O--O-OO---O--OO-O
O--O--OOO-O-O--O----OO-OO-O--O
-----OOOOOOO--O-O--O-OO--OOOO-
OOO-OOO--OOOO--O--OOOOOO--O-OO
-O-O-----O-OOOOOO---O----OO-O-
---OO--O--OOOOO----O-OO-O--OO-
---O-OOO----O-------O-OO-OOO-O
----OOO--OOO---OO--OOOO-O-O-O-
--OOOO-OO-OO-O-OO--OO--OOO--O-
-OO-O-O-O-OO-O-O--------O-----
OO--OO---O--OO-O-O----OO--OOOO
--OOO-OO--OOO----OO-OOO-O-OOOO
printing pluses with volume 2 or larger
(6, 1) 5 (26, 4) 5 (8, 5) 5 (1